#### Mounting S3 bucket to Databricks

- Mount creates a link between a workspace and cloud object storage, which enables you to interact 
with cloud object storage using familiar file paths relative to the Databricks file system.
- To open a new notebook: `New > Notebook`
- Ref: [Databricks notebook](https://dbc-b54c5c54-233d.cloud.databricks.com/?o=1865928197306450#notebook/627262318697111/command/627262318697120)

In [ ]:
# You should see the CSV files you uploaded earlier is now inside the
# FileStore tables folder.
dbutils.fs.ls('/FileStore/tables')

In [ ]:
# PySpark functions.
from pyspark.sql.functions import *
# URL processing.
import urllib

In [ ]:
# Specify file type to be csv.
file_type = 'csv'
# Indicate file has first row as the header.
first_row_is_header = 'true'
# Indicate file has comma as the delimeter.
delimiter = ','
# Read the CSV file to spark dataframe.
aws_keys_df = spark.read.format(file_type) \
    .option('header', first_row_is_header) \
    .option('sep', delimiter) \
    .load('/FileStore/tables/authentication_credentials.csv')

aws_keys_df

In [ ]:
# Get the AWS access key and secret key from the spark dataframe.
ACCESS_KEY = aws_keys_df.where(col('User name')=='databricks-user') \
                        .select('Access key ID') \
                        .collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.where(col('User name')=='databricks-user') \
                        .select('Secret access key') \
                        .collect()[0]['Secret access key']

# Encode the secrete key.
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe='')

In [ ]:
# This cell is not in Databricks as Databricks does not have access to.
# the local credentials.yaml file.
from database_utils import FileReader


creds = FileReader.read('credentials')

In [ ]:
# Mount creates a link between a workspace and cloud object storage,
# which enables you to interact with cloud object storage using familiar
# file paths relative to the Databricks file system.

IAM_USER_NAME = creds['IAM_USER_NAME']

# AWS S3 bucket name.
AWS_S3_BUCKET = f'user-{IAM_USER_NAME}-bucket'
# Mount name for the bucket.
MOUNT_NAME = f'/mnt/{IAM_USER_NAME}-mount'
# Source url.
SOURCE_URL = "s3n://{0}:{1}@{2}" \
                .format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)

# Mount the drive only once.
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

# Optional: to unmount, at the EOF, run: dbutils.fs.unmount(MOUNT_NAME)

In [ ]:
# Check if the S3 bucket was mounted succesfully.
display(dbutils.fs.ls(f'{MOUNT_NAME}/../..'))
display(dbutils.fs.ls(f'{MOUNT_NAME}/..'))
display(dbutils.fs.ls(f'{MOUNT_NAME}/'))

In [ ]:
# Read the JSON format dataset from S3 into Databricks.
# S3 Filepath to pin topic:
# s3://user-<IAM_USER_NAME>-bucket/topics/<IAM_USER_NAME>.pin/partition=0/

# File location and type.
# Asterisk(*) indicates reading all the content of the specified file 
# that have the .json extension.
file_location = f'{MOUNT_NAME}/topics/{IAM_USER_NAME}.pin/partition=0/*.json'
file_type = "json"
# Ask Spark to infer the schema.
infer_schema = "true"
# Read in JSONs from mounted S3 bucket.
df_pin = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .load(file_location)
# Display Spark dataframe to check its content.
display(df_pin)

In [ ]:
# S3 Filepath to geo topic:
# s3://user-<IAM_USER_NAME>-bucket/topics/<IAM_USER_NAME>.geo/partition=0/
file_location = f'{MOUNT_NAME}/topics/{IAM_USER_NAME}.geo/partition=0/*.json'
file_type = "json"
infer_schema = "true"

df_geo = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location)

display(df_geo)

In [ ]:
# S3 Filepath to user topic:
# s3://user-<IAM_USER_NAME>-bucket/topics/<IAM_USER_NAME>.user/partition=0/
file_location = f'{MOUNT_NAME}/topics/{IAM_USER_NAME}.user/partition=0/*.json'
file_type = "json"
infer_schema = "true"

df_user = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location)

display(df_user)

#### Task 1: Clean the Pinterest post DataFrame

In [ ]:
type(df_pin)
df_pin.printSchema()
df_pin.dtypes
df_pin.describe()
df_pin.describe().show()
transformed_df_pin = df_pin

In [ ]:
# Replace empty entries and entries with no relevant data in each column
# with Nones.
# https://www.projectpro.io/recipes/explain-fillna-and-fill-functions-pyspark-databricks
# https://sparkbyexamples.com/pyspark/pyspark-loop-iterate-through-rows-in-dataframe/

from pyspark.sql.functions import col, when


# Define conditions for updating each column.
update_conditions = {
    'description':
        (col('description') == 'No description available Story format', None),
    'follower_count': (col('follower_count') == 'User Info Error', None),
    'image_src': (col('image_src') == 'Image src error.', None),
    'is_image_or_video':
        (~col('is_image_or_video')
         .isin(['image', 'video', 'multi-video(story page format)']), None),
    'poster_name': (col('poster_name') == 'User Info Error', None),
    'tag_list': (col('tag_list') == 'N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e', None),
    'title': (col('title') == 'No Title Data Available', None),
}

# Apply conditional transformations to update multiple columns.
for column, condition in update_conditions.items():
    print(condition)
    transformed_df_pin = transformed_df_pin \
                            .withColumn(column, when(condition[0], condition[1])
                            .otherwise(col(column)))

display(transformed_df_pin)

In [ ]:
# Drop duplicates.
transformed_df_pin = transformed_df_pin.dropDuplicates()
display(transformed_df_pin)

In [ ]:
# Perform the necessary transformations on the follower_count to ensure 
# every entry is a number. Make sure the data type of this column is an
# int.

from pyspark.sql.functions import expr


# Pre-transformation of follower_count.
display(df_pin.select('follower_count').distinct())

transformed_df_pin = transformed_df_pin.withColumn(
    'follower_count',
    expr('CASE WHEN substring(follower_count, -1) = "k" \
               THEN concat(substring(follower_count, 1, \
                    length(follower_count) - 1), "000") \
               WHEN substring(follower_count, -1) = "M" \
               THEN concat(substring(follower_count, 1, \
                    length(follower_count) - 1), "000000") \
               ELSE follower_count \
               END')
)

# Post-transformation of follower_count.
display(transformed_df_pin.select('follower_count').distinct())

In [ ]:
# Convert follower_count to int type.
transformed_df_pin = transformed_df_pin \
    .withColumn('follower_count',col('follower_count').cast('int'))
transformed_df_pin.printSchema()

In [ ]:
# Ensure columns with numbers are of the numeric type:
# downloaded, follower_count, index.
transformed_df_pin.dtypes

In [ ]:
# Clean the data in the save_location column to include only the save 
# location path.
transformed_df_pin = transformed_df_pin.withColumn( \
    'new_save_location', transformed_df_pin.save_location.substr(14, 30))
transformed_df_pin = transformed_df_pin.drop('save_location')
transformed_df_pin = transformed_df_pin \
    .withColumnRenamed('new_save_location', 'save_location')
display(transformed_df_pin)

In [ ]:
# Rename column.
transformed_df_pin = transformed_df_pin.withColumnRenamed('index', 'ind')
transformed_df_pin.columns

In [ ]:
# Rearrange columns.
transformed_df_pin = transformed_df_pin.select(
    'ind', 'unique_id', 'title', 'description', 'follower_count',
    'poster_name', 'tag_list', 'is_image_or_video', 'image_src', 
    'save_location', 'category')
transformed_df_pin.columns

#### Task 2: Clean the geolocation DataFrame

In [ ]:
df_geo.printSchema()
transformed_df_geo = df_geo

In [ ]:
# Drop duplicates.
transformed_df_geo = transformed_df_geo.dropDuplicates()
display(transformed_df_geo)

In [ ]:
# Create a new column coordinates that contains an array based on the
# latitude and longitude columns.
transformed_df_geo = transformed_df_geo.withColumn(
                        'coordinates', array('latitude', 'longitude'))
display(transformed_df_geo)

In [ ]:
# Drop the latitude and longitude columns from the DataFrame.
# OPTIONAL because later on, the columns will be rearranged to exclude 
# the columns in question.
# https://stackoverflow.com/questions/29600673/how-to-delete-columns-in-pyspark-dataframe
# The * is to unpack / destructure the array.
columns_to_drop = ['latitude', 'longitude']
transformed_df_geo = transformed_df_geo.drop(*columns_to_drop)
display(transformed_df_geo)

In [ ]:
# Convert the timestamp column from a string to a timestamp data type.
transformed_df_geo = transformed_df_geo.withColumn(
                        'timestamp', to_timestamp('timestamp.$date')) 
display(transformed_df_geo)

In [ ]:
# Rearrange columns.
transformed_df_geo = transformed_df_geo.select(
                        'ind', 'country', 'coordinates', 'timestamp')
transformed_df_geo.columns

#### Task 3: Clean the user DataFrame

In [ ]:
df_user.printSchema()
transformed_df_user = df_user

In [ ]:
# Drop duplicates.
transformed_df_user = transformed_df_user.dropDuplicates()
display(transformed_df_user)

In [ ]:
# Create a new column user_name that concatenates the information found 
# in the first_name and last_name columns.
transformed_df_user = transformed_df_user.withColumn(
                        'user_name', concat('first_name', 'last_name'))
display(transformed_df_user)

In [ ]:
# Drop the first_name and last_name columns from the DataFrame.
# OPTIONAL because later on, the columns will be rearranged to exclude 
# the columns in question.
columns_to_drop = ['first_name', 'last_name']
transformed_df_user = transformed_df_user.drop(*columns_to_drop)
display(transformed_df_user)

In [ ]:
# Convert the date_joined column from a string to a timestamp data type.
transformed_df_user = transformed_df_user.withColumn(
                        'date_joined', to_timestamp('date_joined.$date')) 
display(transformed_df_user)

In [ ]:
# Rearrange columns.
transformed_df_user = transformed_df_user.select(
                        'ind', 'user_name', 'age', 'date_joined')
transformed_df_user.columns

#### Query the cleaned data

In [ ]:
# Task 4
# Find the most popular Pinterest category people post to based on their 
# country.
# Your query should return a DataFrame that contains the following
# columns:
#   country
#   category
#   category_count, a new column containing the desired query output
# https://sparkbyexamples.com/pyspark/pyspark-orderby-and-sort-explained/
# https://sparkbyexamples.com/pyspark/pyspark-retrieve-top-n-from-each-group-of-dataframe/

df_pin_geo = transformed_df_pin \
                .join(transformed_df_geo, 
                      transformed_df_pin.ind == transformed_df_geo.ind) \
                .select(transformed_df_geo.country,
                        transformed_df_pin.category)

df_category_count = df_pin_geo.groupby('country', 'category').count() \
                        .withColumnRenamed('count','category_count') \
                        .sort('country', 'category_count', 'category',
                              ascending=[True, False, True])

display(df_category_count)

In [ ]:
# Task 4 continued.
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number


window_country = Window.partitionBy('country') \
                    .orderBy(col('category_count').desc())

df_category_count = df_category_count \
                        .withColumn('row', row_number().over(window_country)) \
                        .filter(col('row') == 1) \
                        .drop('row')

display(df_category_count)

In [ ]:
# Task 5: Find which was te most popular category each year.
# Find how many posts each category had between 2018 and 2022.
# Your query should return a DataFrame that contains the following
# columns:
#     post_year, a new column that contains only the year from the
#       timestamp column
#     category
#     category_count, a new column containing the desired query output

df_pin_geo = transformed_df_pin \
                .join(transformed_df_geo,
                      transformed_df_pin.ind == transformed_df_geo.ind) \
                .select(transformed_df_geo.timestamp,
                        transformed_df_pin.category)

df_pin_geo = df_pin_geo \
                .withColumn('post_year', year(df_pin_geo.timestamp)) \
                .filter((col('post_year') >= 2018) & 
                        (col('post_year') <= 2022)) \
                .drop('timestamp')

df_category_count_2018_2022 = df_pin_geo \
                                .groupby('post_year', 'category').count() \
                                .withColumnRenamed('count','category_count') \
                                .sort('post_year', 'category_count', 'category',
                                      ascending=[True, False, True])

display(df_category_count_2018_2022)

In [ ]:
# Task 5 continued.
window_year = Window.partitionBy('post_year') \
                    .orderBy(col('category_count').desc())

df_category_count_2018_2022 = df_category_count_2018_2022 \
                        .withColumn('row', row_number().over(window_year)) \
                        .filter(col('row') == 1) \
                        .drop('row')

display(df_category_count_2018_2022)

In [ ]:
# Task 6: Find the user with the most followers in each country.
# Step 1: For each country find the user with the most followers.
# Your query should return a DataFrame that contains the following columns:
#     country
#     poster_name
#     follower_count

df_pin_geo = transformed_df_pin \
                .join(transformed_df_geo, 
                      transformed_df_pin.ind == transformed_df_geo.ind) \
                .select(transformed_df_geo.country, 
                        transformed_df_pin.poster_name,
                        transformed_df_pin.follower_count)

df_follower_count = df_pin_geo.dropDuplicates()
df_follower_count = df_follower_count \
                        .sort('country', 'follower_count', 'poster_name', 
                              ascending=[True, False, True])

window_country = Window.partitionBy('country') \
                    .orderBy(col('follower_count').desc())
df_follower_count = df_follower_count \
                        .withColumn('row', row_number().over(window_country)) \
                        .filter(col('row') == 1) \
                        .drop('row')

display(df_follower_count)

In [ ]:
# Task 6 continued.
# Step 2: Based on the above query, find the country with the user with 
# most followers.
# Your query should return a DataFrame that contains the following 
# columns:
#     country
#     follower_count
# This DataFrame should have only one entry.

window = Window.orderBy(col('follower_count').desc())
df_max_follower_count = df_follower_count \
                            .withColumn('row', row_number().over(window)) \
                            .filter(col('row') == 1) \
                            .select('country', 'follower_count')

# df_max_follower_count = df_follower_count.agg(max('follower_count'))
display(df_max_follower_count)

In [ ]:
# Task 7: Find the most popular category for different age groups.
# What is the most popular category people post to based on the 
# following age groups:
#     18-24
#     25-35
#     36-50
#     +50

# Your query should return a DataFrame that contains the following
# columns:
#     age_group, a new column based on the original age column
#     category
#     category_count, a new column containing the desired query output

df_pin_user = transformed_df_pin \
                .join(transformed_df_user, 
                      transformed_df_pin.ind == transformed_df_user.ind) \
                .select(transformed_df_user.age, transformed_df_pin.category)

df_category_count_by_age = df_pin_user.withColumn(
                'age_group',
                expr('CASE WHEN age BETWEEN 18 AND 24 THEN "18-24" \
                        WHEN age BETWEEN 25 AND 35 THEN "25-35" \
                        WHEN age BETWEEN 36 AND 50 THEN "36-50" \
                        WHEN age > 50 THEN "50+" \
                        END')
            ).drop('age')

df_category_count_by_age = df_category_count_by_age \
                            .groupBy('age_group', 'category').count() \
                            .withColumnRenamed('count', 'category_count') \
                            .sort('age_group', 'category_count', 'category', 
                                  ascending=[True, False, True])

window_age = Window.partitionBy('age_group') \
                .orderBy(col('category_count').desc())
df_category_count_by_age = df_category_count_by_age \
                            .withColumn('row', row_number().over(window_age)) \
                            .filter(col('row') == 1) \
                            .drop('row')

df_category_count_by_age = df_category_count_by_age \
                            .withColumn(
                                'age_group', 
                                when(col('age_group') == '50+', '+50')
                                    .otherwise(col('age_group')))

display(df_category_count_by_age)

In [ ]:
# Task 8
# What is the median follower count for users in the following age groups:
#     18-24
#     25-35
#     36-50
#     +50

# Your query should return a DataFrame that contains the following columns:
#     age_group, a new column based on the original age column
#     median_follower_count, a new column containing the desired query output

# https://www.educba.com/pyspark-median/
# https://www.machinelearningplus.com/pyspark/pyspark-statistics-median/?utm_content=cmp-true

df_pin_user = transformed_df_pin \
                .join(transformed_df_user,
                      transformed_df_pin.ind == transformed_df_user.ind) \
                .select(transformed_df_user.age,
                        transformed_df_pin.follower_count)

df_median_follower_count_by_age = df_pin_user \
    .withColumn(
        'age_group',
        expr('CASE WHEN age BETWEEN 18 AND 24 THEN "18-24" \
                WHEN age BETWEEN 25 AND 35 THEN "25-35" \
                WHEN age BETWEEN 36 AND 50 THEN "36-50" \
                WHEN age > 50 THEN "50+" \
                END')
    ).select('age_group', 'follower_count') \
        .sort('age_group', 'follower_count')

df_median_follower_count_by_age = df_median_follower_count_by_age \
                                    .na.drop('any')

display(df_median_follower_count_by_age)

In [ ]:
# Task 8 continued.
from pyspark.sql.functions import percentile_approx

# Calculate the median
df_median_follower_count_by_age = df_median_follower_count_by_age \
                                    .groupBy('age_group') \
                                    .agg(percentile_approx('follower_count', 
                                                           0.5) \
                                    .alias('median_follower_count')) \
                                    .sort('age_group')

display(df_median_follower_count_by_age)

In [ ]:
# Task 8 continued
# Replace '50+' with '+50'
df_median_follower_count_by_age = df_median_follower_count_by_age \
                                    .withColumn('age_group',
                                                when(col('age_group') == '50+',
                                                     '+50').otherwise(
                                                         col('age_group')))

display(df_median_follower_count_by_age)

In [ ]:
# Task 9
# Find how many users have joined between 2015 and 2020.
# Your query should return a DataFrame that contains the following 
# columns:
#     post_year, a new column that contains only the year from the 
#       timestamp column
#     number_users_joined, a new column containing the desired query 
#       output

df_joined_users_by_year = transformed_df_user \
                            .select(year('date_joined').alias('post_year')) \
                            .filter((col('post_year') >= 2015) & 
                                    (col('post_year') <= 2020)) \
                            .sort('post_year') \
                            .groupBy('post_year') \
                            .count() \
                            .withColumnRenamed('count', 'number_users_joined')

display(df_joined_users_by_year)

In [ ]:
# Task 10
# Find the median follower count of users have joined between 2015 and 
# 2020.
# Your query should return a DataFrame that contains the following 
# columns:
#     post_year, a new column that contains only the year from the 
#       timestamp column
#     median_follower_count, a new column containing the desired query
#       output

df_pin_user = transformed_df_pin \
                .join(transformed_df_user,
                      transformed_df_pin.ind == transformed_df_user.ind) \
                .select(transformed_df_user.date_joined, 
                        transformed_df_pin.follower_count)

df_median_follower_count_by_year = df_pin_user \
                                    .select(year('date_joined')
                                            .alias('post_year'), 
                                            'follower_count') \
                                    .filter((col('post_year') >= 2015) & 
                                            (col('post_year') <= 2020)) \
                                    .sort('post_year', 'follower_count') \
                                    .na.drop('any')

# Calculate the median
df_median_follower_count_by_year = df_median_follower_count_by_year \
                                    .groupBy('post_year') \
                                    .agg(percentile_approx('follower_count', 
                                                           0.5) \
                                    .alias('median_follower_count')) \
                                    .sort('post_year')

display(df_median_follower_count_by_year)

In [ ]:
# Task 11
# Find the median follower count of users that have joined between 2015 
# and 2020, based on which age group they are part of.
# Your query should return a DataFrame that contains the following 
# columns:
#     age_group, a new column based on the original age column
#     post_year, a new column that contains only the year from the 
#       timestamp column
#     median_follower_count, a new column containing the desired query 
#       output

df_pin_user = transformed_df_pin \
                .join(transformed_df_user, 
                      transformed_df_pin.ind == transformed_df_user.ind) \
                .select(transformed_df_user.age, 
                        transformed_df_user.date_joined, 
                        transformed_df_pin.follower_count)

df_median_follower_count_by_age_and_year = df_pin_user \
    .withColumn(
        'age_group',
        expr('CASE WHEN age BETWEEN 18 AND 24 THEN "18-24" \
                WHEN age BETWEEN 25 AND 35 THEN "25-35" \
                WHEN age BETWEEN 36 AND 50 THEN "36-50" \
                WHEN age > 50 THEN "50+" \
                END')
    ).select('age_group', 
             year('date_joined').alias('post_year'),
             'follower_count') \
        .sort('age_group', 'post_year', 'follower_count') \
        .na.drop('any')

# Calculate the median
df_median_follower_count_by_age_and_year = \
    df_median_follower_count_by_age_and_year \
        .groupBy('age_group', 'post_year') \
        .agg(percentile_approx('follower_count', 0.5) \
        .alias('median_follower_count')) \
        .sort('age_group', 'post_year')

# Replace '50+' with '+50'
df_median_follower_count_by_age_and_year = \
    df_median_follower_count_by_age_and_year \
        .withColumn('age_group',
                    when(col('age_group') == '50+', '+50')
                    .otherwise(col('age_group')))

display(df_median_follower_count_by_age_and_year)